In [1]:
import pandas as pd 
from sklearn import preprocessing 
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer 
from datasets import Dataset 




C:\Users\hp\AppData\Roaming\Python\Python311\site-packages\transformers\utils\generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


# Load dataset

In [2]:
data_path = "../data/jutsus.jsonl"
df = pd.read_json(data_path , lines = True)
df.head()

,jutsu_name,jutsu_type,jutsu_description
0,Adamantine Sealing Chains: Spiral Formation,"Hiden, Ninjutsu, Fūinjutsu, Barrier Ninjutsu, ...",Kushina uses her chains to form a barrier whil...
1,Adamantine Power: Acala,"Kekkei Genkai, Ninjutsu, Taijutsu",Hashirama kicks the opponent away and raises s...
2,Adamantine Prison Wall,"Ninjutsu, Clone Techniques, Bukijutsu","After using Transformation: Adamantine Staff, ..."
3,Adamantine Seal: Monkey Yang Suppression,"Ninjutsu, Fūinjutsu, Cooperation Ninjutsu","After placing fūinjutsu tags in an area, the u..."
4,Acrobat,"Taijutsu, Kenjutsu","The Acrobat (荒繰鷺伐刀, Akurobatto) is a kenjutsu ..."


In [3]:
def simplify_jutsu(jutsu):
    if "Genjutsu" in jutsu :
        return "Genjutsu"
    
    if "Ninjutsu" in jutsu :
        return "Ninjutsu"
    
    if "Taijutsu" in jutsu :
        return "Taijutsu"

In [4]:
df['jutsu_type_simplifeid'] = df['jutsu_type'].apply(simplify_jutsu)


In [5]:
df.head(20)

,jutsu_name,jutsu_type,jutsu_description,jutsu_type_simplifeid
0,Adamantine Sealing Chains: Spiral Formation,"Hiden, Ninjutsu, Fūinjutsu, Barrier Ninjutsu, ...",Kushina uses her chains to form a barrier whil...,Ninjutsu
1,Adamantine Power: Acala,"Kekkei Genkai, Ninjutsu, Taijutsu",Hashirama kicks the opponent away and raises s...,Ninjutsu
2,Adamantine Prison Wall,"Ninjutsu, Clone Techniques, Bukijutsu","After using Transformation: Adamantine Staff, ...",Ninjutsu
3,Adamantine Seal: Monkey Yang Suppression,"Ninjutsu, Fūinjutsu, Cooperation Ninjutsu","After placing fūinjutsu tags in an area, the u...",Ninjutsu
4,Acrobat,"Taijutsu, Kenjutsu","The Acrobat (荒繰鷺伐刀, Akurobatto) is a kenjutsu ...",Taijutsu
5,Accelerated Armed Revolving Heaven,"Kekkei Genkai, Hiden, Ninjutsu, Fūinjutsu, Tai...",Tenten unseals several weapons from her scroll...,Ninjutsu
6,Absorption Sphere,Ninjutsu,"Using the Jutsu Absorption Arm, the user creat...",Ninjutsu
7,Absolute: Fang Passing Fang,"Taijutsu, Collaboration Techniques",Kiba and Akamaru perform the Fang Passing Fang...,Taijutsu
8,1000 Metre Punch,Taijutsu,The user focuses a large amount of chakra into...,Taijutsu
9,16 Hit Combo,Taijutsu,"A very effective move, Ino uses this as a quic...",Taijutsu


In [6]:
df['jutsu_type_simplifeid'].value_counts() 

jutsu_type_simplifeid
Ninjutsu    2255
Taijutsu     397
Genjutsu     101
Name: count, dtype: int64

In [7]:
df['text'] = df['jutsu_name'] + " ." + df['jutsu_description']
df['jutsus'] = df['jutsu_type_simplifeid']
df = df[['text' , 'jutsus']]
df=df.dropna()

In [8]:
df.head()

,text,jutsus
0,Adamantine Sealing Chains: Spiral Formation .K...,Ninjutsu
1,Adamantine Power: Acala .Hashirama kicks the o...,Ninjutsu
2,Adamantine Prison Wall .After using Transforma...,Ninjutsu
3,Adamantine Seal: Monkey Yang Suppression .Afte...,Ninjutsu
4,"Acrobat .The Acrobat (荒繰鷺伐刀, Akurobatto) is a ...",Taijutsu


In [9]:
from bs4 import BeautifulSoup 
from ast import literal_eval 
class Cleaner:
    def __init__(self):
        pass
    
    def put_line_breaks(self , text):
        return text.replace("<\p>" , "<\p>\n ")
    
    def remove_html_tags (self , text):   
        clean_text = BeautifulSoup(text , 'lxml').text
        return clean_text
        
    
    def clean(self , text):
        text = self.put_line_breaks(text)
        text = self.remove_html_tags(text)
        text = text.strip()
        return text 

In [10]:
text_column_name = "text"
label_column_name = "jutsus"

# Clean the text 

In [11]:
cleaner = Cleaner()
df['text_cleaned'] = df[text_column_name].apply(cleaner.clean)


C:\Users\hp\AppData\Local\Temp\ipykernel_15332\1413355162.py:11: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  clean_text = BeautifulSoup(text , 'lxml').text


In [12]:
df.head(2)

,text,jutsus,text_cleaned
0,Adamantine Sealing Chains: Spiral Formation .K...,Ninjutsu,Adamantine Sealing Chains: Spiral Formation .K...
1,Adamantine Power: Acala .Hashirama kicks the o...,Ninjutsu,Adamantine Power: Acala .Hashirama kicks the o...


# Encode Labels 

In [13]:
le = preprocessing.LabelEncoder() 
le.fit(df[label_column_name].tolist())



LabelEncoder()

In [14]:
label_dict  = {index:label_name for index , label_name in enumerate(le.__dict__['classes_'].tolist()) }
label_dict 

{0: 'Genjutsu', 1: 'Ninjutsu', 2: 'Taijutsu'}

In [15]:
df['label']=le.transform(df[label_column_name].tolist())

In [17]:
df.head()

,text,jutsus,text_cleaned,label
0,Adamantine Sealing Chains: Spiral Formation .K...,Ninjutsu,Adamantine Sealing Chains: Spiral Formation .K...,1
1,Adamantine Power: Acala .Hashirama kicks the o...,Ninjutsu,Adamantine Power: Acala .Hashirama kicks the o...,1
2,Adamantine Prison Wall .After using Transforma...,Ninjutsu,Adamantine Prison Wall .After using Transforma...,1
3,Adamantine Seal: Monkey Yang Suppression .Afte...,Ninjutsu,Adamantine Seal: Monkey Yang Suppression .Afte...,1
4,"Acrobat .The Acrobat (荒繰鷺伐刀, Akurobatto) is a ...",Taijutsu,"Acrobat .The Acrobat (荒繰鷺伐刀, Akurobatto) is a ...",2


In [18]:
test_size = 0.2
df_train , df_test =train_test_split(df,
                                     test_size =test_size,
                                     stratify = df['label']
                                    )

In [19]:
df_train['jutsus'].value_counts()

jutsus
Ninjutsu    1804
Taijutsu     317
Genjutsu      81
Name: count, dtype: int64

In [20]:
model_name= "distilbert/distilbert-base-uncased"

In [21]:
tokenizer = AutoTokenizer.from_pretrained(model_name) 

C:\Users\hp\AppData\Roaming\Python\Python311\site-packages\huggingface_hub\file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [22]:
def preprocess_function(tokenizer , examples):
    return tokenizer(examples['text_cleaned'],truncation=True)

In [23]:
train_dataset = Dataset.from_pandas(df_train)
test_dataset = Dataset.from_pandas(df_test)

tokenized_train = train_dataset.map(lambda examples : preprocess_function(tokenizer , examples) ,
                                                                          batched = True)

tokenized_test = test_dataset.map(lambda examples : preprocess_function(tokenizer , examples ),
                                                                          batched = True)

Map:   0%|          | 0/2202 [00:00<?, ? examples/s]

Map:   0%|          | 0/551 [00:00<?, ? examples/s]